## Spam Filtering
In this programming assignment, we will `revisit' Spam Filtering case study with a real data set that has a "label" for every email - i.e. spam or not spam. We will try Logistic Regression, Decision Tree and Random Forests. The assignment goes from data loading to data inspection to data pre-processing to creating a train/test data set to finally doing machine learning, making predictions and evaluating it. This is typically one part of the "full pipeline" in ML modeling/prototyping - So you will get a sampler taste of some "prototype pipeline" work that happens in practice! Have fun!! And if you get stuck somewhere - Use discord - Maybe someone has a suggestion that will unblock you.

## Loading the data set

In [51]:
import pandas as pd


X2_train = pd.read_csv('X1_train.csv',sep=',',header=None,engine='python',error_bad_lines=False)
y2_train = pd.read_csv('y1_train.csv',sep=',',header=None,engine='python',error_bad_lines=False)
X2_test = pd.read_csv('X1_test.csv',sep=',',header=None,engine='python',error_bad_lines=False)
y2_test = pd.read_csv('y1_test.csv',sep=',',header=None,engine='python',error_bad_lines=False)
X2_val = pd.read_csv('X1_val.csv',sep=',',header=None,engine='python',error_bad_lines=False)
y2_val = pd.read_csv('y1_val.csv',sep=',',header=None,engine='python',error_bad_lines=False)

C:\Users\Winston\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


## 1. Data Pre-Processing Step
Follow data processing steps as in previous HW (HW 5 Part 2) - Feel free to add any processing steps that might help with the new models you will try out in this HW - LR and Random Forest.
Also create the train/val/test splits as before (HW 5 part 2) for the emails data set.


In [52]:

X2_val.head()

,0,1
0,tweet_index,tweet
1,2049781431,@Koppite4004 I always get over-excited when th...
2,2186813755,Working on tonight's post
3,2251415926,Gonna try n fall back 2 sleep! Dream of my &qu...
4,2247456751,@danishmarie I remember watching him since ......


In [53]:
from nltk.tokenize import word_tokenize
X2_train['tokenized'] = X2_train[1].apply(lambda x: word_tokenize(x))
X2_val['tokenized'] = X2_val[1].apply(lambda x: word_tokenize(x))
X2_test['tokenized'] = X2_test[1].apply(lambda x: word_tokenize(x))

X2_train.to_csv('X2_train_tokenized.cvs')
X2_val.to_csv('X2_val_tokenized.cvs')
X2_test.to_csv('X2_test_tokenized.cvs')

In [54]:
from nltk.corpus import stopwords
X2_train['filtered_words'] = X2_train['tokenized'].apply(lambda x: [word for word in x if word not in stopwords.words('english')])
X2_val['filtered_words'] = X2_val['tokenized'].apply(lambda x: [word for word in x if word not in stopwords.words('english')])
X2_test['filtered_words'] = X2_test['tokenized'].apply(lambda x: [word for word in x if word not in stopwords.words('english')])

X2_train.to_csv('X2_train_filtered_words.cvs')
X2_val.to_csv('X2_val_filtered_words.cvs')
X2_test.to_csv('X2_test_filtered_words.cvs')


In [55]:
X2_train['new_words'] = X2_train['filtered_words'].apply(lambda x: [word for word in x if word.isalnum()])
X2_val['new_words'] = X2_val['filtered_words'].apply(lambda x: [word for word in x if word.isalnum()])
X2_test['new_words'] = X2_test['filtered_words'].apply(lambda x: [word for word in x if word.isalnum()])

X2_train.to_csv('X2_train_new_words.cvs')
X2_val.to_csv('X2_val_new_words.cvs')
X2_test.to_csv('X2_test_new_words.cvs')



In [56]:
X2_train.head()

,0,1,tokenized,filtered_words,new_words
0,tweet_index,tweet,[tweet],[tweet],[tweet]
1,2048594990,"@mantic59 Cool link, but you got an extra T in...","[@, mantic59, Cool, link, ,, but, you, got, an...","[@, mantic59, Cool, link, ,, got, extra, T, ht...","[mantic59, Cool, link, got, extra, T, htttp, p..."
2,1826490984,@miizluna_jessie tell your mommy i said hiiiii!,"[@, miizluna_jessie, tell, your, mommy, i, sai...","[@, miizluna_jessie, tell, mommy, said, hiiiii...","[tell, mommy, said, hiiiii]"
3,2014223469,Ten o'clock coffee,"[Ten, o'clock, coffee]","[Ten, o'clock, coffee]","[Ten, coffee]"
4,2068667671,ive had some food now i have the sneezies lol,"[ive, had, some, food, now, i, have, the, snee...","[ive, food, sneezies, lol]","[ive, food, sneezies, lol]"


In [57]:
from sklearn.model_selection import train_test_split
train_data = X2_train
rem_data = X2_train.drop(train_data.index).reset_index(drop=True)
val_data = X2_val
test_data = X2_test

# train_data = data_set.sample(frac=0.8, random_state=25).reset_index(drop=True)
# rem_data = data_set.drop(train_data.index).reset_index(drop=True)
# val_data = rem_data.sample(frac=0.5, random_state=25).reset_index(drop=True)
# test_data = rem_data.drop(val_data.index).reset_index(drop=True)

In [58]:
train_data = train_data.drop(['tokenized'], axis = 1)
train_data = train_data.drop(['filtered_words'], axis = 1)
test_data = test_data.drop(['tokenized'], axis = 1)
test_data = test_data.drop(['filtered_words'], axis = 1)
val_data = val_data.drop(['tokenized'], axis = 1)
val_data = val_data.drop(['filtered_words'], axis = 1)

In [59]:
print(train_data.shape)
train_data.head()

(917074, 3)


,0,1,new_words
0,tweet_index,tweet,[tweet]
1,2048594990,"@mantic59 Cool link, but you got an extra T in...","[mantic59, Cool, link, got, extra, T, htttp, p..."
2,1826490984,@miizluna_jessie tell your mommy i said hiiiii!,"[tell, mommy, said, hiiiii]"
3,2014223469,Ten o'clock coffee,"[Ten, coffee]"
4,2068667671,ive had some food now i have the sneezies lol,"[ive, food, sneezies, lol]"


In [60]:
print(val_data.shape)
val_data.head()

(131012, 3)


,0,1,new_words
0,tweet_index,tweet,[tweet]
1,2049781431,@Koppite4004 I always get over-excited when th...,"[Koppite4004, I, always, get, fixture, lists, ..."
2,2186813755,Working on tonight's post,"[Working, tonight, post]"
3,2251415926,Gonna try n fall back 2 sleep! Dream of my &qu...,"[Gon, na, try, n, fall, back, 2, sleep, Dream,..."
4,2247456751,@danishmarie I remember watching him since ......,"[danishmarie, I, remember, watching, since, lo..."


In [61]:
print(test_data.shape)
test_data.head()

(262023, 3)


,0,1,new_words
0,tweet_index,tweet,[tweet]
1,1994530242,@bwatwood i'm at the university of central flo...,"[bwatwood, university, central, florida, orlan..."
2,1975024899,@LouiseGeek LOL he is class thought cool as Ow...,"[LouiseGeek, LOL, class, thought, cool, Owt, r..."
3,2180684802,@MissChriis saad i miss you too i'm bored as ...,"[MissChriis, saad, miss, bored, hell, got, go,..."
4,1684965598,"@KarmaElite @dreamsneverend Yeah, but the girl...","[KarmaElite, dreamsneverend, Yeah, girl, playe..."


## 2. Logistic Regression
Below, try out LR with/without regression on spam filtering. Which model gives you best validation error? What role does regularization play here? Which regularizer works best? Which of these models are more interpretable? And how can you see that? Are any of the issues you saw with Naive Bayes method resolved with LR? What are any limitations you see with your LR model?

In [62]:
print(X2_train.head())

             0                                                  1  \
0  tweet_index                                              tweet   
1   2048594990  @mantic59 Cool link, but you got an extra T in...   
2   1826490984   @miizluna_jessie tell your mommy i said hiiiii!    
3   2014223469                                Ten o'clock coffee    
4   2068667671     ive had some food now i have the sneezies  lol   

                                           tokenized  \
0                                            [tweet]   
1  [@, mantic59, Cool, link, ,, but, you, got, an...   
2  [@, miizluna_jessie, tell, your, mommy, i, sai...   
3                             [Ten, o'clock, coffee]   
4  [ive, had, some, food, now, i, have, the, snee...   

                                      filtered_words  \
0                                            [tweet]   
1  [@, mantic59, Cool, link, ,, got, extra, T, ht...   
2  [@, miizluna_jessie, tell, mommy, said, hiiiii...   
3                       

In [63]:
# Your code here for LR with and without regularization 
# including obtaining validation errors.
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score

y_train = y2_train[1]
y_test = y2_test[1]
y_val = y2_val[1]

cv = CountVectorizer(binary=False, max_df=0.95) 
cv.fit(train_data[1]) # To get a dictionary
train_feature_set = cv.transform(train_data[1])
test_feature_set = cv.transform(test_data[1])
val_feature_set = cv.transform(val_data[1])

def LR():
    clf = LogisticRegression(solver='saga')
    clf.fit(train_feature_set, y_train)

    pred = clf.predict(val_feature_set)
    print('Results of LR')
    print('Accuracy: ', accuracy_score(y_val, pred))
    
    print('aaaa')
    df1 = pd.DataFrame(X2_val)
    df1['Prediction'] = pred
    df1 = df1.set_axis(['a', 'b','c','d','e','Prediction'], axis=1, inplace=False)
    #print(df)
    header = ["a", "Prediction"]
    df1.to_csv('LRout.csv', columns = header, header=False,index=False)
    print('aaaa')

def LR_Regularized():
    clf = LogisticRegression(solver='liblinear', penalty = 'l1')
    clf.fit(train_feature_set, y_train)

    pred = clf.predict(val_feature_set)
    
    print('Results of LR_Regularized')
    print('Accuracy: ', accuracy_score(y_val, pred))
    
    print('aaaa')
    df2 = pd.DataFrame(X2_val)
    df2['Prediction'] = pred
    df2 = df2.set_axis(['a', 'b','c','d','e','Prediction'], axis=1, inplace=False)
    #print(df)
    header = ["a", "Prediction"]
    df2.to_csv('LR_Reg_out.csv', columns = header, header=False,index=False)
    print('aaaa')

def LR_best_hyperparam():
    clf = LogisticRegression(solver='liblinear', penalty = 'l2')
    clf.fit(train_feature_set, y_train)

    pred = clf.predict(val_feature_set)
    
    print('Results of LR_best_hyperparam')
    print('Accuracy: ', accuracy_score(y_val, pred))
    
    print('aaaa')
    df3 = pd.DataFrame(X2_val)
    df3['Prediction'] = pred
    df3 = df3.set_axis(['a', 'b','c','d','e','Prediction'], axis=1, inplace=False)
    #print(df)
    header = ["a", "Prediction"]
    df3.to_csv('LR_best_hyper_out.csv', columns = header, header=False,index=False)
    print('aaaa')

In [64]:
LR()

C:\Users\Winston\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


Results of LR
Accuracy:  0.7970033279394254
aaaa
aaaa


In [65]:
LR_Regularized()

Results of LR_Regularized
Accuracy:  0.7965987848441364
aaaa
aaaa


In [66]:
LR_best_hyperparam()

Results of LR_best_hyperparam
Accuracy:  0.7965835190669557
aaaa
aaaa


## 3. Decision Tree
Train a DT on the same data set. Let depth of the tree be a hyper-param. Which depth gives you the best validation error? What are the top 20 features as per the DT? How do they compare to the top 20 features from LR?

In [67]:
# Your code here for DT
from sklearn.tree import DecisionTreeClassifier

def DT():
    clf = DecisionTreeClassifier()
    clf.fit(train_feature_set, y_train)
    pred = clf.predict(val_feature_set)
    
    print('Results of DT')
    print('Accuracy: ', accuracy_score(y_val, pred))
    
    print('aaaa')
    df4 = pd.DataFrame(X2_val)
    df4['Prediction'] = pred
    df4 = df4.set_axis(['a', 'b','c','d','e','Prediction'], axis=1, inplace=False)
    #print(df)
    header = ["a", "Prediction"]
    df4.to_csv('DT_out.csv', columns = header, header=False,index=False)
    print('aaaa')
    
def DT_best_tree_depth():
    clf = DecisionTreeClassifier(max_depth=60)
    clf.fit(train_feature_set, y_train)
    pred = clf.predict(val_feature_set)
    
    print('Results of DT_best_tree_depth')
    print('Accuracy: ', accuracy_score(y_val, pred))
    
    print('aaaa')
    df5 = pd.DataFrame(X2_val)
    df5['Prediction'] = pred
    df5 = df5.set_axis(['a', 'b','c','d','e','Prediction'], axis=1, inplace=False)
    #print(df)
    header = ["a", "Prediction"]
    df5.to_csv('DT_best_tree_depth_out.csv', columns = header, header=False,index=False)
    print('aaaa')

In [68]:
DT()

Results of DT
Accuracy:  0.7300781607791653
aaaa
aaaa


In [69]:
DT_best_tree_depth()

Results of DT_best_tree_depth
Accuracy:  0.7079962140872592
aaaa
aaaa


## 4. Random Forests
To improve upon DTs you decide to implement a Random Forest. The hyper-params are number of DTs and depth per tree. How does the performance of Random Forest compare against the DT you trained earlier. How does the validation error compare? Also how do the precision/recall compare between DT and Random Forest on test set?

In [70]:
# Your code here for Random Forest
from sklearn.ensemble import RandomForestClassifier

def random_forest():
    clf = RandomForestClassifier(max_depth = 1)
    clf.fit(train_feature_set, y_train)
    pred = clf.predict(val_feature_set)
    
    print('Results of Random Forest')
    print('Accuracy: ', accuracy_score(y_val, pred))
    
    print('aaaa')
    df6 = pd.DataFrame(X2_val)
    df6['Prediction'] = pred
    df6 = df6.set_axis(['a', 'b','c','d','e','Prediction'], axis=1, inplace=False)
    #print(df)
    header = ["a", "Prediction"]
    df6.to_csv('random_forest_out.csv', columns = header, header=False,index=False)
    print('aaaa')
    

def random_forest_hyperparam_tuning():
    clf = RandomForestClassifier(max_depth=10, random_state=42)
    clf.fit(train_feature_set, y_train)
    pred = clf.predict(val_feature_set)
    
    print('Results of Random Forest')
    print('Accuracy: ', accuracy_score(y_val, pred))
    
    print('aaaa')
    df7 = pd.DataFrame(X2_val)
    df7['Prediction'] = pred
    df7 = df7.set_axis(['a', 'b','c','d','e','Prediction'], axis=1, inplace=False)
    #print(df)
    header = ["a", "Prediction"]
    df7.to_csv('random_forest_out.csv', columns = header, header=False,index=False)
    print('aaaa')

def precision_recall_trees():
    clf = DecisionTreeClassifier()
    clf.fit(train_feature_set, y_train)
    pred = clf.predict(test_feature_set)
    
    print('Results of DT')
    print('Precision: ', precision_score(y_test, pred, average='micro'))
    print('Recall: ', recall_score(y_test, pred, average='micro'))
    
    clf = RandomForestClassifier(max_depth = 1)
    clf.fit(train_feature_set, y_train)
    pred = clf.predict(test_feature_set)
    
    print('\nResults of Random Forest')
    print('Precision: ', precision_score(y_test, pred, average='micro'))
    print('Recall: ', recall_score(y_test, pred, average='micro'))
    

In [71]:
random_forest()

Results of Random Forest
Accuracy:  0.58248099410741
aaaa
aaaa


In [72]:
random_forest_hyperparam_tuning()

Results of Random Forest
Accuracy:  0.715132964919244
aaaa
aaaa


In [73]:
precision_recall_trees()

Results of DT
Precision:  0.7298405101842205
Recall:  0.7298405101842205

Results of Random Forest
Precision:  0.5815291024070406
Recall:  0.5815291024070406


## 5. Overall
What are your insights from training these models? Which models are good at what? Which ML model would you use in practice for spam filtering?

I would use Logistic Regression as they yield better results.